### Perform data integration using BBKNN

In [ ]:
import scanpy as sc
import hdf5plugin
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import scanpy.external as sce
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## 8687 genes
H5AD = "./data/4_concatenated_h5ad/lung_cancer_concat_data_v3.h5ad"

# H5AD_INTEGRATED = "./data/6_integrated_h5ads/lung_cancer_bbknn_all_genes.h5ad"

H5AD_INTEGRATED = "./data/6_integrated_h5ads/lung_cancer_bbknn_3000_hvgs.h5ad"

# H5AD_INTEGRATED = "./data/6_integrated_h5ads/lung_cancer_bbknn_top_500_mlc_genes.h5ad"

In [ ]:
IMPORTANT_GENES = "./data/5_csv_dataframes/ctype_condition_multilabel_features_v3.csv"

In [ ]:
def data_preprocessor(adata):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
    adata.raw = adata
    sc.pp.scale(adata, max_value=10)
    sc.pp.pca(adata)
    return adata

In [ ]:
# imp_genes_df = pd.read_csv(IMPORTANT_GENES)
# top500_df = pd.DataFrame(imp_genes_df.loc[:499, 'feature'])
# top500_gene_list = top500_df['feature'].tolist()
# top500_gene_list

In [ ]:
adata_merged = sc.read_h5ad(H5AD)
adata_merged

In [ ]:
# adata_merged = adata_merged[:, adata_merged.var_names.isin(top500_gene_list)]

adata_merged = data_preprocessor(adata_merged)
adata_merged

In [ ]:
sc.external.pp.bbknn(adata_merged, batch_key='batch')

In [ ]:
sc.tl.umap(adata_merged)

In [ ]:
sc.pl.umap(adata_merged, color="batch", legend_fontsize=12, legend_fontoutline=2, 
           title='', show=False)

# sc.pl.umap(adata, color='batch', add_outline=True,
#                legend_fontsize=12, legend_fontoutline=2,frameon=True,
#                title='', palette='Set1', show=False)

plt.savefig('./umap_batches_bbknn_all_genes.png', dpi=600, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata_merged, color="cell_type", legend_fontsize=12, legend_fontoutline=2, 
           title='', show=False)

# sc.pl.umap(adata, color='cell_type', add_outline=True,
#                legend_fontsize=12, legend_fontoutline=2,frameon=True,
#                title='', palette='tab20', show=False)

plt.savefig('./umap_cell_type_bbknn_all_genes.png', dpi=600, bbox_inches='tight')

In [ ]:
sc.pl.umap(adata_merged, color="tissue", legend_fontsize=12, legend_fontoutline=2, 
           title='', show=False)

# sc.pl.umap(adata, color='tissue', add_outline=True,
#                legend_fontsize=12, legend_fontoutline=2,frameon=True,
#                title='', palette='viridis', show=False)

plt.savefig('./umap_conditions_bbknn_all_genes.png', dpi=600, bbox_inches='tight')

In [ ]:
adata_merged.write_h5ad(
    H5AD_INTEGRATED,
    compression=hdf5plugin.FILTERS["zstd"],
    compression_opts=hdf5plugin.Zstd(clevel=5).filter_options
                         )